In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn as sk

In [6]:

files = ["2018.csv", "2019.csv", "2020.csv"]



In [ ]:
def process_file(file_path):
    # Read CSV file
    df = pd.read_csv(file_path)
    
    # Drop completely empty column if it exists
    if 'Unnamed: 27' in df.columns:
        df.drop(columns=['Unnamed: 27'], inplace=True)
    
    # Convert FL_DATE to datetime format
    df['FL_DATE'] = pd.to_datetime(df['FL_DATE'])
    
    # Fill missing numeric values with 0
    df.fillna(0, inplace=True)
    
    return df

# Process each file and store results
dataframes = [process_file(file) for file in files]

# Combine all years into a single DataFrame
df_combined = pd.concat(dataframes, ignore_index=True)

# Display the first few rows
print(df_combined.head())

# Check data info to confirm cleaning
df_combined.info()
